In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import numpy
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
test_df = pd.read_csv("test.csv")
valid_df = pd.read_csv("validation.csv")

In [4]:
train_int = train_df

In [5]:
for index, c in train_int.iterrows():  
    if c.slotformat in "Na":
        train_int.set_value(index, 'slotformat', 10)

In [6]:
d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
train_int['adexchange']=train_int['adexchange'].map(d)

In [7]:
# Set the random seed
np.random.seed(12)

# Initialize label encoder
# label_encoder = preprocessing.LabelEncoder()

In [8]:
for index, c in test_df.iterrows():  
    if c.slotformat in "Na":
        test_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
test_df['adexchange']=test_df['adexchange'].map(d)

In [9]:
for index, c in valid_df.iterrows():  
    if c.slotformat in "Na":
        valid_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
valid_df['adexchange']=valid_df['adexchange'].map(d)

In [10]:
# avgCTR

only_ones = train_int[train_int.click == 1]
avgCTR = len(only_ones)/len(train_int)
avgCTR

0.0007539649884458758

In [11]:
adjust_train = train_int

# Outline

1. predict pCTR  
    * train  
    * validation
    * test    
2. predict Payprice
    * train
    * validation  
    * test      
3. predict BIdprice
    * train  
    * validation  
    * test
4. Evaluation

## First step - Predict pCTR

### (1) Create a model using Train dataset - predict pCTR

In [12]:
adjust_train['area'] = adjust_train['slotwidth']*adjust_train['slotheight']

In [13]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y
X = adjust_train[features]
y = adjust_train['click']

In [14]:
X.head()

,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,1,14,80,85,28080,5,1458,2
1,2,20,94,95,62500,5,3476,2
2,5,9,216,217,94080,0,1458,1
3,2,20,216,221,65520,162,3427,2
4,6,9,40,48,85500,0,3476,1


In [15]:
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
X_dummies = pd.get_dummies(X)

In [17]:
X_dummies = X_dummies.astype("int")

In [18]:
#X_dummies

In [19]:
# Train the model to estimate pCTR
clf_pCTR = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_pCTR.fit(X_dummies, y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [20]:
# pCTR
y_train_pred = clf_pCTR.predict(X_dummies)
y_train_pred

array([  7.00712204e-04,   1.16795301e-04,   5.30719757e-04, ...,
        -7.95722008e-05,   4.37617302e-04,   5.63412905e-04], dtype=float32)

In [21]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_train_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 0.027399584680827743


==> RMSE is small because of the imbalanced dataset. Hence, the small value of RMSE does not mean that the model is good in this case.

### (2) Check a model using Validation dataset - predict pCTR

In [22]:
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

In [23]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y (We use X for predict pCTR, and y for check the error)
X = valid_df[features]
y = valid_df['click']

In [24]:
# Convert datatype into strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [25]:
X_dummies = pd.get_dummies(X)

In [26]:
# To fit a model, convert strings into integer
X_dummies = X_dummies.astype("int")

In [27]:
# predict pCTR by using the model
y_val_pred = clf_pCTR.predict(X_dummies)
y_val_pred

array([ 0.00103101,  0.00782758,  0.00130215, ...,  0.00015113,
        0.00032473,  0.00096089], dtype=float32)

In [28]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_val_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 0.02746000707533313


==> Because of the imbalanceness of the dataset, RMSE is small. I cannot measure the model accuracy from RMSE in this case.

### (3) Apply a model to Test dataset - predict pCTR

In [29]:
test_df['area'] = test_df['slotwidth']*test_df['slotheight']

In [30]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X 
X = test_df[features]

In [31]:
# Convert datatype into strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [32]:
# Create dummies
X_dummies = pd.get_dummies(X)

In [33]:
# To fit a model, convert strings into integer
X_dummies = X_dummies.astype("int")

In [34]:
# predict pCTR by using the model
y_test_pred = clf_pCTR.predict(X_dummies)
y_test_pred

array([ 0.00106522,  0.00012732,  0.00028607, ...,  0.00099769,
        0.00038162,  0.0003221 ], dtype=float32)

In [35]:
pCTR_test = y_test_pred

## Second step - Predict payprice

### (1) Create a model using Train dataset - predict payprice

In [36]:
# predict payprice by using the following features
# "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
# "click" represents pCTR that we calculated above
adjust_train['pCTR'] = y_train_pred

In [37]:
#Define features
features = ["pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y (We use X for predict pCTR, and y for check the error)
X = adjust_train[features]
y = adjust_train['payprice']

In [38]:
X.head()

,pCTR,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,0.000701,1,14,80,85,28080,5,1458,2
1,0.000117,2,20,94,95,62500,5,3476,2
2,0.000531,5,9,216,217,94080,0,1458,1
3,0.000959,2,20,216,221,65520,162,3427,2
4,-0.000054,6,9,40,48,85500,0,3476,1


In [39]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [40]:
X_dummies = pd.get_dummies(X)
X_dummies = X_dummies.astype("int")

In [41]:
# Train the model to estimate bidprice
clf_payprice = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_payprice.fit(X_dummies, y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [42]:
# Payprice
y_train_pay_pred = clf_payprice.predict(X_dummies)
y_train_pay_pred

array([ 64.08774567,  64.3501358 ,  59.76527405, ...,  60.41659164,
        38.14165878,  73.8842926 ], dtype=float32)

In [43]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_train_pay_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 53.48294316856131


In [44]:
check_pay_df = pd.DataFrame([y, y_train_pay_pred]).T

In [45]:
#check_pay_df.head()
check_pay_df.columns = ['payprice', 'predict_payprice']
check_pay_df.astype('float')
gap = check_pay_df['payprice']-check_pay_df['predict_payprice']
gap.mean()

4.276015649494036e-06

### (2) Check a model using Validation dataset - predict payprice

In [46]:
# predict payrice by using the following features
# "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
# "click" represents pCTR that we calculated above
# create the column "pCTR", and put pCTR.
valid_df['pCTR'] = y_val_pred

In [47]:
valid_df.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR
0,0,5,14,91c6a6b9e90c0f54d3230815a5a3e22e,1,u_Vhk7C5STO8TZ3s8,windows_ie,14.122.240.*,216,232,...,1,0,449a22cd91d9042eda3d3a1b89a22ea8,227,102,0f951a030abdaedd733ee8d114ce2944,3427,"10006,10063,10059,10111",75000,0.001031
1,0,3,14,24b3621ad3b063b6c09c541781d534b3,1,u_DANDbCAksMy,android_safari,116.22.55.*,216,217,...,10,118,11908,277,118,null,2997,null,16000,0.007828
2,0,1,13,6e7bce0a9ad42f479827d37bf9618a9d,1,u_VhkxPnKHPo1H3Vb,windows_ie,60.215.242.*,146,149,...,0,70,d01411218cc79bc49d2a4078c4093b76,241,77,b2e35064f3549d447edbbdfb1f707c8c,3427,null,90000,0.001302
3,0,5,18,6873feca381b8293bf418ba8408a014c,1,u_Vh5zL3B83ln9Xbl,windows_chrome,27.193.87.*,146,148,...,0,70,c46090c887c257b61ab1fa11baee91d8,241,77,0f951a030abdaedd733ee8d114ce2944,3427,null,90000,0.001211
4,0,5,14,7e6430333c552cf0b32bb21dc792d2df,1,u_VhKPOr5wOt9HQsj,windows_ie,183.228.214.*,275,275,...,0,5,ff5123fb9333ca095034c62fdaaf51aa,300,178,361e128affece850342293213691a043,3386,null,94080,0.000701


In [48]:
#Define features
features = ["pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y (We use X for predict Payprice, and y for check the error)
X = valid_df[features]
y = valid_df['payprice']

In [49]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [50]:
X_dummies = pd.get_dummies(X)

In [51]:
X_dummies = X_dummies.astype("int")

In [52]:
# Payprice
y_valid_pay_pred = clf_payprice.predict(X_dummies)
y_valid_pay_pred

array([ 121.86728668,  133.04492188,   82.28263855, ...,   71.50524902,
         58.57446289,   62.5828743 ], dtype=float32)

In [53]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_valid_pay_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 53.55377873895582


In [54]:
check_pay_df = pd.DataFrame([y, y_valid_pay_pred]).T

In [55]:
#check_pay_df.head()
check_pay_df.columns = ['payprice', 'predict_payprice']
check_pay_df.astype('float')
gap = check_pay_df['payprice']-check_pay_df['predict_payprice']
gap.mean()

0.0342409603625537

### (3) Check a model using Validation dataset - predict payprice

In [56]:
# predict payprice by using the following features
# "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
# "click" represents pCTR that we calculated above
# create the column "pCTR", and put pCTR.
test_df['pCTR'] = y_test_pred

In [57]:
#Define features
features = ["pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X 
X = test_df[features]

In [58]:
X.head()

,pCTR,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,0.001065,0,17,80,85,75000,0,3427,1
1,0.000127,5,22,1,1,90000,80,1458,3
2,0.000286,2,13,3,8,90000,20,3427,3
3,0.000350,4,17,40,46,94080,0,3386,1
4,0.000189,6,20,146,151,90000,31,3476,3


In [59]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [60]:
X_dummies = pd.get_dummies(X)

In [61]:
X_dummies = X_dummies.astype("int")

In [62]:
# Payprice
y_test_pay_pred = clf_payprice.predict(X_dummies)
y_test_pay_pred

array([ 126.35311127,   82.78099823,   44.54524231, ...,  147.65446472,
         81.44208527,  116.61724091], dtype=float32)

# Third step - predict bidprice

### (1) Create a model using Train dataset - predict bidprice

In [63]:
# predict bitprice by using the following features
# "predicted payprice", "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
adjust_train['pPayprice'] = y_train_pay_pred

In [64]:
adjust_train.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR,pPayprice
0,0,1,14,fdfae6789b787899f1b875de3ab8b21a,1,u_Vh1OPkFv3q5CFdR,windows_ie,180.107.112.*,80,85,...,5,612599432d200b093719dd1f372f7a30,300,54,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063",28080,0.000701,64.087746
1,0,2,20,372883147cdefbc495fb5082f79dde9b,1,u_VhkRL6dCOTLsq-c,windows_chrome,125.120.199.*,94,95,...,5,8dff45ed862a740986dbe688aafee7e5,249,42,15b749127478946f161a54dc0dad27c8,3476,"10063,10111",62500,0.000117,64.350136
2,0,5,9,2912cb5947e1fad450e2473c2f61ef75,1,u_VhkyORMrD8TP3GL,windows_ie,120.196.64.*,216,217,...,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,bebefa5efe83beee17a3d245e7c5085b,1458,10063,94080,0.000531,59.765274
3,0,2,20,d7b7d61d1e50e1d27d210928d28a9b61,1,u_VhkyLRqVL2a4eWL,windows_ie,219.131.122.*,216,221,...,162,7184c9560e68e977187e67e45a4f3198,238,182,0f951a030abdaedd733ee8d114ce2944,3427,"10077,10111",65520,0.000959,185.056381
4,0,6,9,657d317dc4045bb79c2652abbea07530,1,u_Vh1zOJuQOtTCBgL,windows_chrome,123.190.78.*,40,48,...,0,c938195f9e404b4f38c7e71bf50263e5,238,34,a3becb967c015e9acb5e72f22b100ef8,3476,"10031,13042,10006,10057,13866,10063,14273,1005...",85500,-0.000054,81.921036


In [65]:
#Define features
features = ["pPayprice", "pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y
X = adjust_train[features]
y = adjust_train['bidprice']

In [66]:
X.head()

,pPayprice,pCTR,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,64.087746,0.000701,1,14,80,85,28080,5,1458,2
1,64.350136,0.000117,2,20,94,95,62500,5,3476,2
2,59.765274,0.000531,5,9,216,217,94080,0,1458,1
3,185.056381,0.000959,2,20,216,221,65520,162,3427,2
4,81.921036,-0.000054,6,9,40,48,85500,0,3476,1


In [67]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [68]:
X_dummies = pd.get_dummies(X)

In [69]:
X_dummies = X_dummies.astype("int")

In [70]:
# Train the model to estimate bidprice
clf_bidprice = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_bidprice.fit(X_dummies, y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [71]:
# Bidprice
y_train_bid_pred = clf_bidprice.predict(X_dummies)
y_train_bid_pred

array([ 300.00994873,  249.03451538,  299.9876709 , ...,  249.0466156 ,
        240.98391724,  299.99597168], dtype=float32)

In [72]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_train_bid_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 0.09503523239111931


### (2) Check a model using Validation dataset - predict bidprice

In [73]:
# predict bitprice by using the following features
# "predict payprice", "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
valid_df['pPayprice'] = y_valid_pay_pred

In [74]:
valid_df.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR,pPayprice
0,0,5,14,91c6a6b9e90c0f54d3230815a5a3e22e,1,u_Vhk7C5STO8TZ3s8,windows_ie,14.122.240.*,216,232,...,0,449a22cd91d9042eda3d3a1b89a22ea8,227,102,0f951a030abdaedd733ee8d114ce2944,3427,"10006,10063,10059,10111",75000,0.001031,121.867287
1,0,3,14,24b3621ad3b063b6c09c541781d534b3,1,u_DANDbCAksMy,android_safari,116.22.55.*,216,217,...,118,11908,277,118,null,2997,null,16000,0.007828,133.044922
2,0,1,13,6e7bce0a9ad42f479827d37bf9618a9d,1,u_VhkxPnKHPo1H3Vb,windows_ie,60.215.242.*,146,149,...,70,d01411218cc79bc49d2a4078c4093b76,241,77,b2e35064f3549d447edbbdfb1f707c8c,3427,null,90000,0.001302,82.282639
3,0,5,18,6873feca381b8293bf418ba8408a014c,1,u_Vh5zL3B83ln9Xbl,windows_chrome,27.193.87.*,146,148,...,70,c46090c887c257b61ab1fa11baee91d8,241,77,0f951a030abdaedd733ee8d114ce2944,3427,null,90000,0.001211,81.311287
4,0,5,14,7e6430333c552cf0b32bb21dc792d2df,1,u_VhKPOr5wOt9HQsj,windows_ie,183.228.214.*,275,275,...,5,ff5123fb9333ca095034c62fdaaf51aa,300,178,361e128affece850342293213691a043,3386,null,94080,0.000701,68.958977


In [75]:
#Define features
features = ["pPayprice", "pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y (We use X for predict pCTR, and y for check the error)
X = valid_df[features]
y = valid_df['bidprice']

In [76]:
X.head()

,pPayprice,pCTR,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,121.867287,0.001031,5,14,216,232,75000,0,3427,1
1,133.044922,0.007828,3,14,216,217,16000,118,2997,0
2,82.282639,0.001302,1,13,146,149,90000,70,3427,3
3,81.311287,0.001211,5,18,146,148,90000,70,3427,3
4,68.958977,0.000701,5,14,275,275,94080,5,3386,2


In [77]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [78]:
X_dummies = pd.get_dummies(X)

In [79]:
X_dummies = X_dummies.astype("int")

In [80]:
# Bidprice
y_valid_bid_pred = clf_bidprice.predict(X_dummies)
y_valid_bid_pred

array([ 226.94418335,  276.91467285,  240.99215698, ...,  237.98127747,
        238.11968994,  299.98745728], dtype=float32)

In [81]:
# Mean square errors ==> the measurement of errors
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_valid_bid_pred ,y)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE))

RMSE: 0.09508061099565136


### (3)  Apply a model to Test dataset - predict bidprice

In [82]:
# predict bitprice by using the following features
# "predicted payprice", "click", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"
test_df['pPayprice'] = y_test_pay_pred

In [83]:
#Define features
features = ["pPayprice", "pCTR", "weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X 
X = test_df[features]

In [84]:
X.head()

,pPayprice,pCTR,weekday,hour,region,city,area,slotprice,advertiser,adexchange
0,126.353111,0.001065,0,17,80,85,75000,0,3427,1
1,82.780998,0.000127,5,22,1,1,90000,80,1458,3
2,44.545242,0.000286,2,13,3,8,90000,20,3427,3
3,75.070518,0.000350,4,17,40,46,94080,0,3386,1
4,39.484688,0.000189,6,20,146,151,90000,31,3476,3


In [85]:
# convert int datatype to strings in order to create dummies
X['advertiser']=X['advertiser'].astype(str)
X['adexchange']=X['adexchange'].astype(str)

/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nishimuraaki/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [86]:
X_dummies = pd.get_dummies(X)

In [87]:
X_dummies = X_dummies.astype("int")

In [88]:
# Bidprice
y_test_bid_pred = clf_bidprice.predict(X_dummies)
y_test_bid_pred

array([ 226.97390747,  300.00396729,  240.98391724, ...,  227.00663757,
        293.99832153,  293.92584229], dtype=float32)

# Evaluation

### (1) validation dataset

In [89]:
# predicted CTR
pCTR_valid = y_val_pred
pCTR_valid

array([ 0.00103101,  0.00782758,  0.00130215, ...,  0.00015113,
        0.00032473,  0.00096089], dtype=float32)

In [90]:
# predicted average CTR
avg_pCTR = sum(pCTR_valid)/len(pCTR_valid)
avg_pCTR

0.00075543436506681719

In [91]:
# Actual average CTR in validation dataset
only_ones = valid_df[valid_df.click == 1]
actual_avgCTR = len(only_ones)/len(valid_df)
actual_avgCTR

0.0007539641500055046

==> avg_pCTR and actual avgCTR are close to each other.

In [92]:
valid_df['pBidprice'] = y_valid_bid_pred
valid_df.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR,pPayprice,pBidprice
0,0,5,14,91c6a6b9e90c0f54d3230815a5a3e22e,1,u_Vhk7C5STO8TZ3s8,windows_ie,14.122.240.*,216,232,...,449a22cd91d9042eda3d3a1b89a22ea8,227,102,0f951a030abdaedd733ee8d114ce2944,3427,"10006,10063,10059,10111",75000,0.001031,121.867287,226.944183
1,0,3,14,24b3621ad3b063b6c09c541781d534b3,1,u_DANDbCAksMy,android_safari,116.22.55.*,216,217,...,11908,277,118,null,2997,null,16000,0.007828,133.044922,276.914673
2,0,1,13,6e7bce0a9ad42f479827d37bf9618a9d,1,u_VhkxPnKHPo1H3Vb,windows_ie,60.215.242.*,146,149,...,d01411218cc79bc49d2a4078c4093b76,241,77,b2e35064f3549d447edbbdfb1f707c8c,3427,null,90000,0.001302,82.282639,240.992157
3,0,5,18,6873feca381b8293bf418ba8408a014c,1,u_Vh5zL3B83ln9Xbl,windows_chrome,27.193.87.*,146,148,...,c46090c887c257b61ab1fa11baee91d8,241,77,0f951a030abdaedd733ee8d114ce2944,3427,null,90000,0.001211,81.311287,240.984558
4,0,5,14,7e6430333c552cf0b32bb21dc792d2df,1,u_VhKPOr5wOt9HQsj,windows_ie,183.228.214.*,275,275,...,ff5123fb9333ca095034c62fdaaf51aa,300,178,361e128affece850342293213691a043,3386,null,94080,0.000701,68.958977,300.015472


### - Within the budget

In [93]:
# predicted Clicks - Validation dataset
price_payed = 0
clicked_won = 0 
won = 0
for i in range(len(valid_df)): 
    if (valid_df["pBidprice"][i] > valid_df["pPayprice"][i]):
        price_payed = price_payed +  valid_df["pPayprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            avg_pCTR_valid = valid_df['pCTR'][:i].mean()
            if (valid_df['pCTR'][i] > 0.05):
                clicked_won = clicked_won + 1
        else:
            break
            
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )
print("average pCTR: ", avg_pCTR_valid)

With budget 6250 we won  80  auctions and on  0  was clicked
average pCTR:  0.0008391508017666638


In [94]:
# Spend (predict)
# predicted Clicks - Validation dataset
price_payed = 0
clicked_won = 0 
won = 0
for i in range(80): 
    if (valid_df["pBidprice"][i] > valid_df["pPayprice"][i]):
        price_payed = price_payed +  valid_df["pPayprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            avg_pCTR_valid = valid_df['pCTR'][:i].mean()
            if (valid_df['pCTR'][i] > 0.05):
                clicked_won = clicked_won + 1
        else:
            break

print("Predict spend:", price_payed)
# print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )
# print("average pCTR: ", avg_pCTR_valid)

Predict spend: 6241.06651878


In [95]:
# Actual
price_payed = 0
clicked_won = 0 
won = 0
for i in range(len(valid_df)): 
    if (valid_df["bidprice"][i] > valid_df["payprice"][i]):
        price_payed = price_payed +  valid_df["payprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            #avg_pCTR_valid = valid_df["click"][:i]/len(valid_df["click"][:i])
            if (valid_df['click'][i] == 1):
                clicked_won = clicked_won + 1
        else:
            break
            
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )
#print("average pCTR: ", avg_pCTR_valid)

With budget 6250 we won  75  auctions and on  0  was clicked


In [96]:
# Actual Spend
# Actual
price_payed = 0
clicked_won = 0 
won = 0
for i in range(76): 
    if (valid_df["bidprice"][i] > valid_df["payprice"][i]):
        price_payed = price_payed +  valid_df["payprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            avg_pCTR_valid = valid_df['pCTR'][:i].mean()
            if (valid_df['click'][i] == 1):
                clicked_won = clicked_won + 1
        else:
            break
print("Actual spend:", price_payed)        
# print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )
print("average pCTR: ", avg_pCTR_valid)

Actual spend: 6237
average pCTR:  0.0008292158599942923


### (2) Test dataset

In [97]:
# from step1(3)
pCTR_test = y_test_pred
pCTR_test

array([ 0.00106522,  0.00012732,  0.00028607, ...,  0.00099769,
        0.00038162,  0.0003221 ], dtype=float32)

In [98]:
# CTR
avg_pCTR = sum(pCTR_test)/len(pCTR_test)
avg_pCTR

0.00075485314287559232

In [99]:
test_df['pBidprice'] = y_test_bid_pred

In [100]:
# Clicks -Testdataset
price_payed = 0
clicked_won = 0 
won = 0
for i in range(len(test_df)): 
    if (test_df["pBidprice"][i] > test_df["pPayprice"][i]):
        price_payed = price_payed +  test_df["pPayprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            avg_pCTR_test = test_df['pCTR'][:i].mean()
            if (test_df['pCTR'][i] > 0.05):
                clicked_won = clicked_won + 1
        else:
            break
            
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )
print("average pCTR: ", avg_pCTR_test)

With budget 6250 we won  77  auctions and on  0  was clicked
average pCTR:  0.0009124777279794216


In [101]:
# Clicks -Testdataset
price_payed = 0
clicked_won = 0 
won = 0
for i in range(77): 
    if (test_df["pBidprice"][i] > test_df["pPayprice"][i]):
        price_payed = price_payed +  test_df["pPayprice"][i]
        
        if (price_payed < 6250):
            won = won + 1
            if (test_df['pCTR'][i] > 0.05):
                clicked_won = clicked_won + 1
        else:
            break
            
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, " was clicked" )

With budget 6250 we won  77  auctions and on  0  was clicked


In [102]:
price_payed

6199.9463500976562

In [103]:
len((test_df["pPayprice"] < test_df["pBidprice"]))

299749

In [104]:
len(test_df)

299749

In [105]:
# Clicks (Number of clicked bids)
only_win_click = test_df[((test_df["pPayprice"] < test_df["pBidprice"]) & (test_df["pCTR"] > 0.05))]
len(only_win_click)

5

In [106]:
## CTR for only_win
only_win = test_df[(test_df["pPayprice"] < test_df["pBidprice"])]
CTR_win = len(only_win_click)/len(only_win)
print('CTR_win:', CTR_win)

CTR_win: 1.6683739693619804e-05


In [107]:
# Compare payprice and bidprice, and if bidprice is higher than payprice, sum the payprice.
# Total spend
test_df.loc[(test_df["pPayprice"] < test_df["pBidprice"]), 'pPayprice'].sum()

24027514.0

In [108]:
# avg CPM ... (cost /# of impressions)
print('avg CPM:', sum(only_win['pPayprice'])/len(test_df))

avg CPM: 80.1587961409


In [109]:
# avg CPC ... (cost/# click)
print('avg CPC:', sum(only_win['pPayprice']/len(only_win_click)))

avg CPC: 4805503.79736
